In [232]:
import pandas as pd
import datetime
import re
import time

In [233]:
# df1 = pd.read_csv('CSL.AX001.csv')
# df2 = pd.read_csv('CSL.AX002.csv')

In [234]:
df1 = pd.read_excel(r'New folder\WES.AX001.xlsx')
df2 = pd.read_excel(r'New folder\WES.AX002.xlsx')

Fill the empty Data Column with Previous Cell Value

In [235]:
df1.Date = df1.Date.fillna(method='ffill')
df1 = df1.fillna('')

Converting into Datetime Object

In [236]:
df1.Date = pd.to_datetime(df1.Date)
df2.Year = pd.to_datetime(df2.Year, format=r'%Y-%m-%d')

In [237]:
def remove_str(row):
    x = row.Comments2
    try:
        x = x.replace(re.search('Originally posted .*', x)[0], '').strip()
    except:
        pass
    try:
        x = x.replace(re.search('Release Date:.*', x)[0], '').strip()
    except:
        pass
    row.Comments2 = x
    return row
df1 = df1.apply(remove_str, axis='columns')

Previous Year's Number of Post

In [238]:
def prev_year(date_temp):
    return df1[(((df1.Date - date_temp).astype('timedelta64[h]')/24) >= -365) & (((df1.Date - date_temp).astype('timedelta64[h]')/24) <=0)].shape[0]

90 days post number

In [239]:
def num_of_post(filtered_df):
    date_90_len = filtered_df.shape[0]
    return date_90_len

Remove Unwanted Lines from Comments

In [240]:
# filtered_df = filtered_df.apply(remove_str, axis='columns')

Row containing at least one negative words

In [241]:
def row_containing_neg_words(filtered_df, keyword):
    def func_inner(row):      
        for each_word in keyword:
            if each_word in str(row.Comments2):
                return 1
        return 0
    return sum(filtered_df.apply(func_inner, axis='columns'))

Number of Negative Words Mentioned

In [242]:
def criticism_words(filtered_df, keyword):
    def func2(row):
        word_count = 0
        for each in keyword:
            word_count += len(re.findall(each.lower(), str(row.Comments2)))
        return word_count
    return sum(filtered_df.apply(func2, axis='columns'))

Total Number of Words

In [243]:
def words_length(filtered_df):
    def func3(row):
        return len(str(row.Comments2).split())
    return sum(filtered_df.apply(func3, axis='columns'))

Output

In [244]:
def main_function(row): 
    keyword = ['am', 'is', 'are', 'was', 'were']
    row['Total Post in a Financial Year'] = prev_year(row['Year'])
    filtered_df = df1[(abs((df1.Date - row['Year']).astype('timedelta64[h]')/24) <= 90)]
    # print(filtered_df.isna().any())
    row['Total post related to earnings'] = num_of_post(filtered_df)
    row['Criticism Comments'] = row_containing_neg_words(filtered_df, keyword)
    row['Criticism Words'] = criticism_words(filtered_df, keyword)
    row['Total no. of words'] = words_length(filtered_df)
    return row
df2 = df2.apply(main_function, axis='columns')

In [245]:
df2.to_csv('3.csv')